In [1]:
#necessary library
library(data.table) #fread
library(lubridate)
library(reshape2)
library(dplyr)
library(scales)
library(stringr)
library(tidyr) # spread function
#library(tidyverse)

Warning message:
"package 'data.table' was built under R version 3.6.3"

Attaching package: 'lubridate'


The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following object is masked from 'package:base':

    date


Warning message:
"package 'reshape2' was built under R version 3.6.3"

Attaching package: 'reshape2'


The following objects are masked from 'package:data.table':

    dcast, melt


Warning message:
"package 'dplyr' was built under R version 3.6.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:lubridate':

    intersect, setdiff, union


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'scales' was built under R 

In [3]:
# Select the month you want to investigate
YEARLIST =("19")
MONTHLIST = c("04","06")
DISTANCE_FILEPATH = "../../data/tidy/vehicle-trajectory-computation/"
COMPUTATION_FILEPATH = "../../data/tidy/"
energy_df = fread("../../data/raw/energy-consumption-08-20.csv") # Read in energy data
d_ridership = fread("../../data/raw/ridership-2019-2020.csv")# Read in ridership data

# Bin variables creation and aggregate table

In [27]:
# aggregrate_trajectory_table
line_aggregation = function(year,month){
    assign("dg",fread(paste0(DISTANCE_FILEPATH, paste(paste("green", "trajectory", year, month, sep = "-", collapse = ""), ".csv", sep=""))))
    assign("dh",fread(paste0(DISTANCE_FILEPATH, paste(paste("heavy", "trajectory", year, month, sep = "-", collapse = ""), ".csv", sep=""))))
    dg$lineid = 4
    dg = subset(dg, select = c(trxtime, year, month, day, lineid, lat, lon , speed_kph , accel_mps2 , interval_seconds , dist_meters , vehicleid))
    dh = subset(dh, select = c(trxtime, year, month, day, lineid, lat, lon , speed_kph , accel_mps2 , interval_seconds , dist_meters , vehicleid))
    df = rbind(dg, dh) 
    return(df)
}

In [33]:
# Unit conversion
unit_transfer = function(df){
   df$hour = hour(df$trxtime)
   df$speed_mph = df$speed_kph*0.621371 #kph to mph
   df$distance_mile = df$dist_meters*0.000621371 #convert from meters to mile
   df$time_hr = df$interval_seconds/3600.0 #convert from seconds to hour
  return(df)
}    

In [11]:
# Calculate the speed bins 
bin_speeds <- function (dataframe, num_bins) {
  dataframe = data.table(dataframe)
  dummy_cols = c(paste0("speed_bin_",1:num_bins,"_dummy"))
  bin_time_cols = c(paste0("speed_bin_",1:num_bins,"_time_hr"))
  cutpoints <- quantile(dataframe$speed_mph,seq(0, 1, 1/num_bins),na.rm=TRUE) 
  print(paste0("The speed bins are: ", cutpoints))
  for(n in seq(1, num_bins)) {
    if(n == 1){
     dataframe[, dummy_cols[n] := sapply(speed_mph,  function(x) ifelse (x < cutpoints[n+1], 1, 0))]
    }
    else if (n == num_bins){
     dataframe[, dummy_cols[n] := sapply(speed_mph,  function(x) ifelse (x >= cutpoints[num_bins], 1, 0))]
    }
    else {
    dataframe[, dummy_cols[n] := sapply(speed_mph,  function(x) ifelse (x >= cutpoints[n] & x < cutpoints[n + 1], 1, 0))]
    }
  }
 dataframe[, (bin_time_cols) := lapply(.SD, function(x) x * dataframe$time_hr ), .SDcols = dummy_cols]
 print(paste0("Percentage error of summed speed bin times = ", 
              round(100*(sum(colSums(dataframe %>% select(starts_with("speed_bin_") & ends_with("_time_hr")),na.rm=TRUE)) 
                         - sum(dataframe$time_hr, na.rm=TRUE))/sum(dataframe$time_hr, na.rm=TRUE),2), "%"))
 return(dataframe) 
}

In [12]:
# Calculate the acceleration bins 
bin_accelerations <- function (dataframe, num_bins) {
  dataframe = data.table(dataframe)
  dummy_cols = c(paste0("accel_bin_",1:num_bins,"_dummy"))
  bin_time_cols = c(paste0("accel_bin_",1:num_bins,"_time_hr"))
  cutpoints <- quantile(dataframe$accel_mps2,seq(0, 1, 1/num_bins),na.rm=TRUE) 
  print(paste0("The acceleration bins are: ", cutpoints))
  for(n in seq(1, num_bins)) {
    if(n == 1){
      dataframe[, dummy_cols[n] := sapply(accel_mps2,  function(x) ifelse (x < cutpoints[n+1], 1, 0))]
    }
    else if (n == num_bins){
      dataframe[, dummy_cols[n] := sapply(accel_mps2,  function(x) ifelse (x >= cutpoints[num_bins], 1, 0))]
    }
    else {
      dataframe[, dummy_cols[n] := sapply(accel_mps2,  function(x) ifelse (x >= cutpoints[n] & x < cutpoints[n + 1], 1, 0))]
    }
  }
  dataframe[, (bin_time_cols) := lapply(.SD, function(x) x * dataframe$time_hr ), .SDcols = dummy_cols]
  print(paste0("Percentage error of summed acceleration bin times = ", 
               round(100*(sum(colSums(dataframe %>% select(starts_with("accel_bin_") & ends_with("_time_hr")),na.rm=TRUE)) 
                          - sum(dataframe$time_hr, na.rm=TRUE))/sum(dataframe$time_hr, na.rm=TRUE),2), "%"))                                                  
  return(dataframe)
}

In [15]:
# add speed-acceleration bin interaction terms
bin_interaction_terms = function(df,num_bins){
    df = data.frame(df)
    for (i in seq(1, num_bins)){
        for (j in seq(1,num_bins)){
            # add interaction dummy variables
            df = data.frame(df)
            dummy_cols_interaction = paste0("speed_bin_",i,"_accel_bin_",j,"_dummy")
            speed_dummy = paste0("speed_bin_",i,"_dummy")
            accel_dummy = paste0("accel_bin_",i,"_dummy")
            df[[dummy_cols_interaction]] = df[[speed_dummy]]*df[[accel_dummy]]
            # add interaction terms
            df = data.table(df)
            bin_times_cols_interaction = paste0("speed_bin_",i,"_accel_bin_",j,"_time_hr")
            df[,(bin_times_cols_interaction) := lapply(.SD, function(x) x * df$time_hr ), .SDcols = dummy_cols_interaction]
        }
    }
  return(df)
}

In [39]:
# Aggregate dataframe at hour level
hour_aggregate <- function (dataframe,num_bins) {
    dataframe = data.table(dataframe)
    dataframe$month = as.character(dataframe$month)
    dataframe$hour = as.character(dataframe$hour)
    dataframe$day = as.character(dataframe$day)
    # create another data table to summary the number of trains running in each hour
    d_num_trains <- dataframe[, c("month",'hour',"day","lineid","vehicleid")]
    agg_d_num_trains = d_num_trains[, .(count = length(unique(vehicleid))), by = .(month,day,hour,lineid)]
    agg_d_num_trains_wide = spread(agg_d_num_trains, lineid,count)
   # interaction term name preparation for aggregating by hour
    speed_name = paste0("speed_bin_",1:num_bins)
    accel_name = paste0("_","accel_bin_",1:num_bins,"_time_hr")
    interaction_name = outer(speed_name,accel_name, paste, sep="")
    # aggregate by hour
    sum_cols = c("distance_mile","time_hr",paste0("speed_bin_",1:num_bins,"_time_hr"),paste0("accel_bin_",1:num_bins,"_time_hr"),interaction_name)
    agg_df = dataframe[, lapply( .SD, sum , na.rm=TRUE), by = c("year","month",'hour',"day"), .SDcols = sum_cols]
    avg_interval_speed_mph_df = dataframe[, lapply( .SD, mean , na.rm=TRUE), by = c("year","month","hour","day"), .SDcols = 'speed_mph']
    agg_df[, 'avg_interval_speed_mph'] = avg_interval_speed_mph_df$speed_mph
    agg_df[, 'avg_hour_speed_mph'] = agg_df$distance_mile/agg_df$time_hr
    merged_agg_df = merge(agg_d_num_trains_wide,agg_df,all=T) 
    return( merged_agg_df)
}

In [81]:
# Combine ridership data
merge_ridership = function(merged_dt,d_ridership){
   merged_dt$year = as.character(merged_dt$year)
   d_ridership$year = as.character(year(d_ridership$servicedate))
   d_ridership$month = as.character(month(d_ridership$servicedate))
   d_ridership$day = as.character(day(d_ridership$servicedate))
   d_ridership$hour = as.character(hour(d_ridership$halfhour))
   d_ridership = d_ridership[,sum(rawtaps_split),by = .(year, month, day,hour)] 
   names(d_ridership)[names(d_ridership) == 'V1'] <- 'ridership'
   merged_db = merge(merged_dt,d_ridership,by = c("year","month","day","hour"),all=F) 
   return(merged_db)
}

In [85]:
# Combine energy consumption data
merge_energy <- function (energy_df,hour_dt) {
    # Melt by hour 
    melted_energy_df = melt(energy_df, id.vars=c('Year','Month','Day of Month','WJ','TAVG'), measure.vars = paste0("Hour ",1:24))
        colnames(melted_energy_df) = c('year', 'month', 'day', 'weekends', 'TAVG', 'Hour', 'energy_MWh')
    hour_energy_dt <- setDT(melted_energy_df)
    hour_energy_dt[, Hour := str_replace(Hour, "Hour ", "")]
    hour_energy_dt$Hour = as.numeric(hour_energy_dt$Hour)
    hour_energy_dt$Hour = hour_energy_dt$Hour - 1
    colnames(hour_energy_dt)[6] <- 'hour'
    hour_energy_dt$year = as.character(hour_energy_dt$year)
    hour_energy_dt$month = as.character(hour_energy_dt$month)
    hour_energy_dt$day = as.character(hour_energy_dt$day)
    hour_energy_dt$hour = as.character(hour_energy_dt$hour)
    merged_dt = merge(hour_dt,hour_energy_dt,by = c("year","month","day","hour") , all = F)
    return(merged_dt)
}

## Computing the final table

In [8]:
main <- function (num_bins,energy_df,d_ridership,YEARLIST,MONTHLIST) {
    for (y in YEARLIST) {
        for (m in MONTHLIST) {
             interval_df = line_aggregation(y,m)
             interval_agg <- interval_df %>% unit_transfer（) %>% bin_speeds(num_bins) %>% 
             bin_accelerations(num_bins) %>% bin_interaction_terms(num_bins) %>% hour_aggregate(num_bins)
             # merge with ridership
             merge_ridership = merge_ridership(interval_agg,d_ridership)
             # merge with energy table
             merge_energy = merge_energy(energy_df, merge_ridership)
             write.csv(merge_final,file.path(paste0(COMPUTATION_FILEPATH, paste(paste("green", "trajectory", "aggregation" , y, m , sep = "-", collapse = ""), ".csv", sep=""))))
            }
      }
   
}

In [9]:
main(8,energy_df,d_ridership,YEARLIST,MONTHLIST)

   V1             trxtime trainid vehicleid routeid counter lineid      lat
1:  1 2019-04-18 05:33:20   10001      3653       0   25599      4 42.33696
2:  2 2019-04-18 05:50:21   10001      3653       0   27350      4 42.33689
3:  3 2019-04-18 05:51:52   10001      3653       0   27585      4 42.33692
4:  4 2019-04-18 05:52:04   10001      3653       0   27624      4 42.33666
5:  5 2019-04-18 07:25:46   10002      3653     852   48005      4 42.33506
6:  6 2019-04-18 07:25:53   10002      3653     852   48030      4 42.33443
         lon servicetypeid sourceid car1 car2 car3 speed heading
1: -71.25329             1        1 3653   NA   NA  1.97   25.19
2: -71.25384             1        1 3653   NA   NA  6.09   24.92
3: -71.25150             1        1 3653   NA   NA 15.29  123.31
4: -71.25081             1        1 3653   NA   NA 20.47  124.89
5: -71.24777             1        1 3653 3842   NA 33.42  124.03
6: -71.24654             1        1 3653 3842   NA 33.57  123.81
             